In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Set paths
# Working from the Scripts folder as the base path
exercise_path = '../Python/Exercise Files/Pandas Course Resources/Pandas Course Resources/retail/'

### Aggregation
`df_data.groupby('col1')[['do_sometthing_col']].mean()` - this will return a data frame, single `[ ]` will return a series

In [3]:
# Load transactions data
transactions = pd.read_csv(os.path.join(exercise_path, "transactions.csv"), parse_dates=['date'])

In [ ]:
# Group transactions by store and return the top 10 stores
transactions.groupby('store_nbr')[['transactions']].sum().sort_values(['transactions'], ascending=False).head(10)

,transactions
store_nbr,
44,7273093
47,6535810
45,6201115
46,5990113
3,5366350
48,5107785
8,4637971
49,4574103
50,4384444


In [12]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [ ]:
#transactions = transactions.assign(month = transactions.date.dt.month)
#transactions.info()

transactions["month"] = transactions["date"].dt.month

transactions.head()
#transactions.groupby(['store_nbr', 'month'])[['transactions']].sum().sort_values(by = ['month', 'transactions'], ascending=[True, False])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
 3   month         83488 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(2)
memory usage: 2.2 MB


,date,store_nbr,transactions,month
0,2013-01-01,25,770,1
1,2013-01-02,1,2111,1
2,2013-01-02,2,2358,1
3,2013-01-02,3,3487,1
4,2013-01-02,4,1922,1


In [28]:
# Wrap code in parath to have cleaner looking code
(transactions
 .groupby(['store_nbr', 'month'])[['transactions']]
 .sum()
 .sort_values(by = ['month', 'transactions'], ascending=[True, False]
 ).reset_index() # Convert multicolumns to indices)
).head(10)

,store_nbr,month,transactions
0,44,1,628438
1,47,1,568824
2,45,1,538370
3,46,1,522763
4,3,1,463260
5,48,1,439045
6,8,1,404463
7,49,1,386589
8,50,1,372093
9,11,1,336187


In [29]:

grouped = (
    transactions.groupby(["store_nbr", "month"])
    .agg({"transactions": ["sum", "mean"]})
    .sort_values(by=["month", ("transactions", "sum")], ascending=[True, False])
)

In [30]:
grouped

transactions             
                         sum         mean
store_nbr month                          
44        1           628438  4246.202703
47        1           568824  3843.405405
45        1           538370  3637.635135
46        1           522763  3532.182432
3         1           463260  3151.428571
...                      ...          ...
32        12           86167   718.058333
21        12           84128  1402.133333
42        12           76741  1279.016667
29        12           76627  1277.116667
22        12           50650   844.166667

[641 rows x 2 columns]

In [31]:
grouped.reset_index()

store_nbr month transactions             
                             sum         mean
0          44     1       628438  4246.202703
1          47     1       568824  3843.405405
2          45     1       538370  3637.635135
3          46     1       522763  3532.182432
4           3     1       463260  3151.428571
..        ...   ...          ...          ...
636        32    12        86167   718.058333
637        21    12        84128  1402.133333
638        42    12        76741  1279.016667
639        29    12        76627  1277.116667
640        22    12        50650   844.166667

[641 rows x 4 columns]

In [36]:
grouped.loc[(3, 1)]

transactions  sum     463260.000000
              mean      3151.428571
Name: (3, 1), dtype: float64

In [38]:
grouped.iloc[4]

transactions  sum     463260.000000
              mean      3151.428571
Name: (3, 1), dtype: float64

In [ ]:
grouped.droplevel(0, axis = 1).reset_index().head(10)

store_nbr month transactions             
                           sum         mean
0        44     1       628438  4246.202703
1        47     1       568824  3843.405405
2        45     1       538370  3637.635135
3        46     1       522763  3532.182432
4         3     1       463260  3151.428571
5        48     1       439045  2966.520270
6         8     1       404463  2751.448980
7        49     1       386589  2612.087838
8        50     1       372093  2514.141892
9        11     1       336187  2286.986395

In [41]:
grouped.droplevel(0, axis = 1).reset_index().head(10)

,store_nbr,month,sum,mean
0,44,1,628438,4246.202703
1,47,1,568824,3843.405405
2,45,1,538370,3637.635135
3,46,1,522763,3532.182432
4,3,1,463260,3151.428571
5,48,1,439045,2966.520270
6,8,1,404463,2751.448980
7,49,1,386589,2612.087838
8,50,1,372093,2514.141892
9,11,1,336187,2286.986395
